In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [26]:
from src.utils import (
    load_data,
    remove_duplicates_in_coloumn,
    convert_from_degree_to_ciruclar,
    resample_hourly,
    create_time_features,
    find_repeated_indices,
    load_val_dates,
    create_lag_features
)

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [11]:
predictions_1 = pd.read_csv("../data/comparison/model_1_train_predictions.csv")
predictions_2 = pd.read_csv("../data/comparison/train_predictions.csv")

train_dates = pd.read_csv("../data/processed/train2.csv")

In [44]:
scaler = MinMaxScaler(feature_range=(0, 365))

train_day_of_year = scaler.fit_transform(train_dates[["dayofyear"]])
train_dates["day_of_year"] = train_day_of_year
predictions_1["day_of_year"] = train_day_of_year
predictions_2["day_of_year"] = train_day_of_year

In [50]:
val_dates = load_val_dates()
days_of_year = list(set([datetime.strptime(date, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday for date in val_dates]))
winter_days_of_year = [day - 100 for day in days_of_year]

In [38]:
target_predictions_1 = predictions_1[predictions_1["day_of_year"].isin(days_of_year)]["predict"]
target_predictions_2 = predictions_2[predictions_2["day_of_year"].isin(days_of_year)]["predict"]

In [43]:
train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"]

0        0.673973
1        0.673973
2        0.676712
3        0.676712
4        0.676712
           ...   
81161    0.326027
81162    0.326027
81163    0.326027
81164    0.326027
81165    0.326027
Name: dayofyear, Length: 81166, dtype: float64

### Model Comparison for Kaggle Days

In [45]:
mean_absolute_error(target_predictions_1, train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"])

0.12727934615643224

In [46]:
mean_absolute_error(target_predictions_2, train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"])

0.1325528024583385

### Model Comparison for All Days

In [53]:
mean_absolute_error(
    predictions_1["predict"],
    train_dates["pv_measurement"],
)

0.05651121393456173

In [54]:
mean_absolute_error(
    predictions_2["predict"],
    train_dates["pv_measurement"],
)

0.061755802813476396

### Model Comparison for Winter Days

In [51]:
mean_absolute_error(
    predictions_1[predictions_1["day_of_year"].isin(winter_days_of_year)]["predict"],
    train_dates[train_dates["day_of_year"].isin(winter_days_of_year)]["pv_measurement"],
)

0.02724627429429478

In [52]:
mean_absolute_error(
    predictions_2[predictions_2["day_of_year"].isin(winter_days_of_year)]["predict"],
    train_dates[train_dates["day_of_year"].isin(winter_days_of_year)]["pv_measurement"],
)

0.032483011367746166